The system
=======
A system consists mainly of three parts:
    - Entities. May represent agents or inactive entities like physical space.
    - Links. Represent connections between the entities. 
    - Time. The passage of time happens.
    
Basically a system is one big graph that passes information through the links between the nodes.

In [160]:
class System:
    """
    The global system and event dispatcher.

    Aware of hosts, space, and the passage of time (steps). It is
    the main struture of the program, being the only observable and given it
    coordinates all processes.

    """

    def __init__(self):
        'Initialize an empty system.'
        self.entities = set()
        self.entitynames = {}
        self.links = {}
        self.time = None
    # ---

    def addEntity(self, entity, name):
        'Add an entity to the graph.'
        self.entities.add(entity)
        self.entitynames[name] = entity
    # ---
        
    def step(self):
        'Take a single step forward in time.'
        # Process linked items
        #TODO...
        # Process each entity
        for entity in self.entities:
            entity.process(self.time)
    # ---
        
        
    def run(self, steps):
        'Start running the simulation.'
        # Init time
        if self.time is None:
            self.time = 0
        # Take steps
        for _ in range(steps):
            self.step()
    # ---
    
    def stateof(self, entityname):
        "Ask the entity for it's state"
        entity = self.entitynames[entityname]
        return entity.logstate()
    # ---
                
                

Entities
=====
An entity is something that resides in the system. 

It processes information according to it's internal state and the information flowing through the links it shares with other entities.

In [161]:
class Entity:
    '''An entity is something that resides in the system.

    It processes information according to it's internal state 
    and the information flowing through the links it shares with 
    other entities.
    
    An entity registers the following methods:
        - process(time)
        - logstate()
    '''
    
    def process(self, time):
        pass
    
    def logstate(self):
        pass
    
# --- Entity

In [162]:
# < --- Expected use case:

class Cells(Entity):
    'A special type of group entity that follows a Fibbonacci pattern'
    
    def __init__(self):
        self.current = 1
        self.previous = 0
        self.changes = {}
    
    def process(self, time):
        c = self.current
        p = self.previous
        self.current, self.previous = c+p, c
        print('Processed: ', self.logstate())
        
    def logstate(self):
        return f'Current cells: {self.current}'
# --- Cells


# Create a system
sys = System()

# Add the cells
sys.addEntity(Cells(), 
              name='cells')

# Run the processes in the system
sys.run(steps=10)

# Get the state of the cells
sys.stateof('cells')

Processed:  Current cells: 1
Processed:  Current cells: 2
Processed:  Current cells: 3
Processed:  Current cells: 5
Processed:  Current cells: 8
Processed:  Current cells: 13
Processed:  Current cells: 21
Processed:  Current cells: 34
Processed:  Current cells: 55
Processed:  Current cells: 89


'Current cells: 89'

Interactions
=======

Multiple entities can interact between them through Interactions.

Interactions link 2 or more entities to perform a series of actions together.

In [163]:
class Interaction:
    'A structure representing flow of information btw entities.'
    
    def __init__(self, entities, effect):
        self.entities = entities
        self.effects = [effect]
        
    def append(self, effect):
        'Effects btw the same entities can be appended and executed in order'
        self.effects.append(effect)
# --- Interaction    

In order to achieve this, the system has to have aditional control for the propagation of the information.

In [164]:
class System:
    """
    The global system and event dispatcher.

    Aware of hosts, space, and the passage of time (steps). It is
    the main struture of the program, being the only observable and given it
    coordinates all processes.

    """

    def __init__(self):
        'Initialize an empty system.'
        self.entities = set()
        self.toentity = {}
        self.toentityname = {}
        
        self.interactions = set()
        self.time = None
    # ---

    def addEntity(self, entity, name):
        'Add an entity to the graph.'
        self.entities.add(entity)
        self.toentity[name] = entity
        self.toentityname[entity] = name
    # ---
        
    def step(self):
        'Take a single step forward in time.'
        # Process linked items
        for inter in self.interactions:
            entities = inter.entities
            for effect in inter.effects:
                effect(*entities)
        # Process each entity
        for entity in self.entities:
            entity.process(self.time)
    # ---
         
    def run(self, steps):
        'Start running the simulation.'
        # Init time
        if self.time is None:
            self.time = 0
        # Take steps
        for _ in range(steps):
            self.step()
    # ---
    
    def stateof(self, entityname):
        "Ask the entity for it's state"
        entity = self.toentity[entityname]
        return entity.logstate()
    # ---
    
    def link(self, effect, entitynames):
        'Add a link between two named entities'
        entities = tuple(self.toentity[ename] 
                                for ename in entitynames)
        
        # Check if there is already some link btw
        # those same entities.
        if entities in self.interactions:
            # Add the new effect
            interactions[entities].append(effect)
            
        else:
            # Else, add the new link
            interaction = Interaction(entities, effect)
            self.interactions.add( interaction )
    # ---
# --- System
                

Below we show a simple use case where interactions play an important role.

In [165]:
# < --- Intended use case:

class Animals(Entity):
    'A simple group entity.'
    
    def __init__(self, pop=10):
        self.hunt_results = None
        self.population = pop
    # ---
        
    @property
    def population(self):
        return self._population
    # ---
    
    @population.setter
    def population(self, value):
        'The population consists of integers >= 0'
        if value < 0:
            self._population = 0
        else:
            self._population = int(value)
    # ---
                                   
    def logstate(self):
        return self.population
    # ---
# ---

def limit(quantity, interval):
    bottom, top = interval
    if quantity > top:
        quantity = top
        
    elif quantity < bottom:
        quantity = bottom
        
    return quantity
# ---
        
    
class Predators(Animals):
    def process(self, time):
        self.population += self.hunt_results
        self.hunt_results = None
        self.die()
        self.logstate()
        
    def die(self):
        self.population -= 0.3*self.population
        
    def logstate(self):
        print('Total predators: ', self.population)
# ---

class Preys(Animals):
    def process(self, time):
        # Reduce population due to hunting
        self.population -= self.hunt_results
        self.hunt_results = None
        self.reproduce()
        self.logstate()
        
    def logstate(self):
        print('Total preys: ', self.population)
        
    def reproduce(self):
        self.population *= 2
# ---
            

def predation(predators, preys):
    '''The predation interaction.
    
    h (hunting) is the effect of predators in prey.
    f (feeding) is the effect of prey in predators.
    '''
    h, f = 0.005, 0.0002
    close_encounters = preys.population * predators.population
    
    preys.hunt_results = h * close_encounters
    predators.hunt_results = limit(f * close_encounters,
                                   [0, predators.population])
    
    print(f'Predators are hunting {int(preys.hunt_results)} preys')
# ---
    

# Create the system
sys = System()

# Create the cells
sys.addEntity(Predators(pop=100),
              name='Predators')
sys.addEntity(Preys(pop=100),
              name='Preys')

# Add the link btw the cells
sys.link(predation, 
         entitynames=['Predators', 'Preys'])

# Verify the initial state
sys.stateof('Predators')
sys.stateof('Preys')

# Run the simulation
sys.run(steps=50)

Total predators:  100
Total preys:  100
Predators are hunting 50 preys
Total predators:  71
Total preys:  100
Predators are hunting 35 preys
Total predators:  50
Total preys:  128
Predators are hunting 32 preys
Total predators:  35
Total preys:  192
Predators are hunting 33 preys
Total predators:  25
Total preys:  316
Predators are hunting 39 preys
Total predators:  18
Total preys:  552
Predators are hunting 49 preys
Total predators:  13
Total preys:  1004
Predators are hunting 65 preys
Total predators:  10
Total preys:  1876
Predators are hunting 93 preys
Total predators:  9
Total preys:  3564
Predators are hunting 160 preys
Total predators:  10
Total preys:  6806
Predators are hunting 340 preys
Total predators:  14
Total preys:  12930
Predators are hunting 905 preys
Total predators:  19
Total preys:  24048
Predators are hunting 2284 preys
Total predators:  26
Total preys:  43526
Predators are hunting 5658 preys
Total predators:  36
Total preys:  75734
Predators are hunting 13632 prey

The use case we are mainly interested in is in simulating several cell types along with cellular automata fields for chemicals, so we proceed to move forwards towards that intended use case

Simple Cellular Automaton
===============

Now we tackle the problem of a simple 1D CA within our framework.

In [166]:
class CellularAutomaton(Entity):
    'A simple cellular automaton.'
    
    def __init__(self, n=25):
        # Place a single active cell in the middle
        self.gridstate = tuple(1 if i == n//2 else 0
                                   for i in range(n))
        self.rule = [0, 1, 0, 1, 1, 0, 1, 0]
        
    def process(self, time):
        self.gridstate = self.applyrule(self.gridstate, self.rule)
        print(''.join(self.logstate()))
        
    def applyrule(self, grid, rule):
        newgrid = list(grid)
        for i,cell in enumerate(grid):
            # Cells at the edges are constant
            if 1 <= i < len(grid)-1:
                neighbors = grid[i-1:i+2]
                newgrid[i] = self.rule[self.neighbor_n(neighbors)]
        return tuple(newgrid)
                
    def neighbor_n(self, neighbors):
        a, b, c = neighbors
        return c + b*2 + a*4
    
    def logstate(self):
        return [ '#' if cell else '.' for cell in self.gridstate ]
    
sys = System()

sys.addEntity(CellularAutomaton(80),
              name='Sierpinsky')

print( ''.join(sys.stateof('Sierpinsky')) )

sys.run(steps=50)

........................................#.......................................
.......................................#.#......................................
......................................#...#.....................................
.....................................#.#.#.#....................................
....................................#.......#...................................
...................................#.#.....#.#..................................
..................................#...#...#...#.................................
.................................#.#.#.#.#.#.#.#................................
................................#...............#...............................
...............................#.#.............#.#..............................
..............................#...#...........#...#.............................
.............................#.#.#.#.........#.#.#.#............................
............................

We note now (with the contortions in the developer side on `CellularAutomaton.process`, line 12 and on the user side in the line 35, only to print the system state), that it is important to be available to get a representation (as __repr__ and __str__) of the system itself, as for the entities and interactions, as well as to expose the real state of the entities. So, we make further changes to the system.

In [145]:
class Entity:
    '''An entity is something that resides in the system.

    It processes information according to it's internal state 
    and the information flowing through the links it shares with 
    other entities.
    
    An entity registers the following methods:
        - process(time)
    '''
    def __init__(self):
        self.state = None
    
    def process(self, time):
        pass
# --- Entity



class System:
    """
    The global system and event dispatcher.

    Aware of hosts, space, and the passage of time (steps). It is
    the main struture of the program, being the only observable and given it
    coordinates all processes.

    """

    def __init__(self):
        'Initialize an empty system.'
        self.entities = set()
        self.toentity = {}
        self.toentityname = {}
        
        self.interactions = set()
        self.time = None
    # ---
    
    def __getitem__(self, entityname):
        'Access the entities by name.'
        return self.toentity[entityname]
    # ---

    def add(self, entity, name):
        'Add an entity to the graph.'
        self.entities.add(entity)
        self.toentity[name] = entity
        self.toentityname[entity] = name
    # ---
        
    def step(self):
        'Take a single step forward in time.'
        # Process linked items
        for inter in self.interactions:
            entities = inter.entities
            for effect in inter.effects:
                effect(*entities)
        # Process each entity
        for entity in self.entities:
            entity.process(self.time)
    # ---
    
        
    def start(self):
        'Initialize things to be initialized.'
        # Init time
        if self.time is None:
            self.time = 0
    # ---
         
    def run(self, steps):
        'Start running the simulation.'
        # Make sure things are initialized 
        self.start()
        
        # Take steps
        for _ in range(steps):
            self.step()
    # ---
    
    def stateof(self, entityname):
        "Ask the entity for it's state"
        return self[entityname].state
    # ---
    
    def link(self, effect, entitynames):
        'Add a link between named entities.'
        entities = tuple(self.toentity[ename] 
                                for ename in entitynames)
        
        # Check if there is already some link btw
        # those same entities.
        if entities in self.interactions:
            # Add the new effect
            interactions[entities].append(effect)
            
        else:
            # Else, add the new link
            interaction = Interaction(entities, effect)
            self.interactions.add( interaction )
    # ---
# --- System

In [173]:
# Intended use case...
class CellularAutomaton(Entity):
    'A simple cellular automaton.'
    
    def __init__(self, n=25):
        # Place a single active cell in the middle
        self.gridstate = tuple(1 if i == n//2 else 0
                                   for i in range(n))
        self.rule = [0, 1, 0, 1, 1, 0, 1, 0]
    # ---
        
    @property
    def state(self):
        return [ '#' if cell else '.' for cell in self.gridstate ]
    # ---
        
    def process(self, time):
        'Move one time step forward.'
        self.gridstate = self.applyrule(self.gridstate)
        print(''.join(self.state))
    # ---
        
    def applyrule(self, grid):
        'Apply automaton rule to the given grid.'
        newgrid = list(grid)
        for i,cell in enumerate(grid):
            # Cells at the edges are constant
            if 1 <= i < len(grid)-1:
                neighbors = grid[i-1:i+2]
                newgrid[i] = self.rule[self.process_neighbor(neighbors)]
        # Return an immutable new grid
        return tuple(newgrid)
                
    def process_neighbor(self, neighbors):
        'Convert neighborhood to integer to apply rule.'
        a, b, c = neighbors
        return c + b*2 + a*4
    
    def __str__(self):
        return ''.join(self.state)
# --- CellularAutomaton
    

# < --- ---    


sys = System()

sys.add(CellularAutomaton(80),
              name='Sierpinsky')

print(sys['Sierpinsky'] )

sys.run(steps=50)

........................................#.......................................
.......................................#.#......................................
......................................#...#.....................................
.....................................#.#.#.#....................................
....................................#.......#...................................
...................................#.#.....#.#..................................
..................................#...#...#...#.................................
.................................#.#.#.#.#.#.#.#................................
................................#...............#...............................
...............................#.#.............#.#..............................
..............................#...#...........#...#.............................
.............................#.#.#.#.........#.#.#.#............................
............................

We made an important change to the Entity object interface, as it can be seen, the method `logstate` was eliminated, and now the Entity state can and should be accessed with the `state` attribute. From this, now 
```python
    sys.stateof('SomeEntity') == sys['SomeEntity'].state == sys.toentity['SomeEntity'].state
```

Now, it is desirable to have the ability to modify the run behavior on the fly, so, one option is to allow for hooks in the run function and on adding the automaton.

In [182]:
import collections

class Interaction:
    'A structure representing flow of information btw entities.'
    
    def __init__(self, entities, effect):
        self.entities = entities
        self.effects = [effect]
        
    def append(self, effect):
        'Effects btw the same entities can be appended and executed in order'
        self.effects.append(effect)
        
    def process(self):
        'Executes the interaction.'
        for effect in self.effects:
            effect(*self.entities)
# --- Interaction 



class System:
    """
    The global system and event dispatcher.

    Aware of hosts, space, and the passage of time (steps). It is
    the main struture of the program, being the only observable and given it
    coordinates all processes.

    """

    def __init__(self):
        'Initialize an empty system.'
        self.entities = set()
        self.toentity = {}
        self.toentityname = {}
        
        self.interactions = {}
        self.time = None
        
        self.inithooks = {}
        self.prehooks = {}
        self.hooks = {}
    # ---
    
    def __getitem__(self, entityname):
        'Access the entities by name.'
        return self.toentity[entityname]
    # ---

    def add(self, entity, name, inithook=None):
        'Add an entity to the graph.'
        self.entities.add(entity)
        self.toentity[name] = entity
        self.toentityname[entity] = name
        # Process hooks
        if inithook:
            self.add_interaction_to(self.inithooks,
                                    inithook, 
                                    [name])
    # ---
    
    def process_interactions_in(self, interactions):
        'From the dict-like container of interactions, process items.'
        for interaction in interactions.values():
            interaction.process()
    # ---
        
    def step(self):
        'Take a single step forward in time.'
        # Process pre-step hooks
        self.process_interactions_in(self.prehooks)
        
        # Process linked items
        self.process_interactions_in(self.interactions)
        # Process each entity
        for entity in self.entities:
            entity.process(self.time)
            
        # Process post-step hooks
        self.process_interactions_in(self.hooks)
    # ---
         
    def start(self):
        'Initialize things before starting simulation.'
        # Init time
        if self.time is None:
            self.time = 0
        # Make the initialization actions
        self.process_interactions_in(self.inithooks)
    # ---
    
    def update_hooks(self, hooktype, newhooks):
        'Add the given hooks to the system.'
        if newhooks:
            for hook, entitynames in newhooks.items():
                self.add_interaction_to(hooktype, hook, entitynames)
    # ---
         
    def run(self, steps, inithooks=None, hooks=None, prehooks=None):
        'Start running the simulation.'
        # Handle hooks
        self.update_hooks(self.inithooks, inithooks)
        self.update_hooks(self.hooks, hooks)
        self.update_hooks(self.prehooks, prehooks)
        
        # Make sure things are initialized 
        self.start()
        
        # Take steps
        for _ in range(steps):
            self.step() 
    # ---
    
    def stateof(self, entityname):
        "Ask the entity for it's state"
        return self[entityname].state
    # ---
    
    def link(self, effect, entitynames):
        'Add an interation btw named entities.'
        self.add_interaction_to(self.interactions,
                                effect, 
                                entitynames)
    # ---
    
    def add_interaction_to(self, container, effect, entitynames):
        'Add an interaction btw named entities to a dict-like container.'
        entities = tuple(self.toentity[ename] 
                                for ename in entitynames)
        
        # Check if there is already some link btw
        # those same entities.
        if entities in container:
            # Add the new effect
            container[entities].append(effect)
            
        else:
            # Else, add the new link
            interaction = Interaction(entities, effect)
            container[entities] = interaction
    # ---
# --- System

In [183]:
# Intended use case...
class SilentCA(CellularAutomaton):
    'A cellular automaton that does not print to the screen.'
        
    def process(self, time):
        'Move one time step forward, do not print state.'
        self.gridstate = self.applyrule(self.gridstate)
    # ---
# --- CellularAutomaton


def simplelogging(entity):
    'We will use this to print the state.'
    print(entity)
# ---


## < --- --- ----


sys = System()

sys.add(SilentCA(100), 
        name='Sierpinsky')

sys.run(steps=50, 
        prehooks={simplelogging:['Sierpinsky']})

print('Final state: ', sys['Sierpinsky'])
print(sys.stateof('Sierpinsky'))

..................................................#.................................................
.................................................#.#................................................
................................................#...#...............................................
...............................................#.#.#.#..............................................
..............................................#.......#.............................................
.............................................#.#.....#.#............................................
............................................#...#...#...#...........................................
...........................................#.#.#.#.#.#.#.#..........................................
..........................................#...............#.........................................
.........................................#.#.............#.#...............................

In [154]:
sys

In [155]:
sys.entities

{<__main__.SilentCA at 0x7f28e423a2b0>}

In [156]:
for e in sys.entities:
    help(e)

Help on SilentCA in module __main__ object:

class SilentCA(CellularAutomaton)
 |  A cellular automaton that does not print to the screen.
 |  
 |  Method resolution order:
 |      SilentCA
 |      CellularAutomaton
 |      Entity
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  process(self, time)
 |      Move one time step forward, do not print state.
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from CellularAutomaton:
 |  
 |  __init__(self, n=25)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  applyrule(self, grid)
 |      Apply automaton rule to the given grid.
 |  
 |  process_neighbor(self, neighbors)
 |      Convert neighborhood to integer to apply rule.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from CellularAutomaton:
 |  
 |  state
 |  
 |  -----------------

We see now that several changes have been done to the code. The hooks where added with three flavors: inithooks (executed before the simulation starts), prehooks (executed before any step) and hooks (executed at the end of every step). This developed in a major factorization of how the interactions are applied, as the hooks are implemented using the same mechanism.

Now, let's modify the cellular automaton object to be painlessly 1D, 2D or 3D and possibly n-dimensional.

In [344]:
# Helper function
import random as rnd

def random_trial(probability):
    'Return True with a given probability, otherwise return False.'
    # Get a random real btw 0 and 1
    selector = rnd.random()
    
    return (selector < probability)
# ---

In [710]:
from functools import reduce

class CellularAutomaton:
    'An extensible cellular automaton.'
    
    # Default location of the neighbors
    neighborhood = ( (-1,-1), (-1, 0), (-1, 1),
                     ( 0,-1),          ( 0, 1),
                     ( 1,-1), ( 1, 0), ( 1, 1) )
    
    def __init__(self, *args, **kwargs):
        'Create the automaton with the specified dimensions.'
        self.dimensions = kwargs.get('dimensions', (50, 50))
        initialdensity = kwargs.get('initialdensity', 0.5)
        # Create a linear grid
        self.cells = [int(random_trial(initialdensity)) for i in range(self.totalcells)]
    # ---
    
    def rule(self, cell_coords, cell, neighbors):
        'Get next state according to the cell and neighbors'
        alive_neighbors = sum(neighbors)
        # Game of life rules
        if cell:
            return 1 if (1 < alive_neighbors < 4) else 0
        else:
            return 1 if (alive_neighbors == 3) else 0
    # ---
    
    def __str__(self):
        string = ''
        for row in self.state:
            string += ''.join( '#' if cell else '.' for cell in row )
            string += '\n'
        return string
    
    # ---
    
    @property
    def state(self):
        rows, cols = self.dimensions
        state = [ [self.at((i,j)) for j in range(cols) ] for i in range(rows) ]
        return state
    # ---
    
    ### The following is NOT likely to be overriden by subclasses
    
    def neighborsof(self, position):
        'Get the coordinates of the neighboring cells.'
        try:
            # Check if the positions have been already calculated
            return self.neighbors[position]
        
        except AttributeError:
            # Generate neighbors
            self.neighbors = {pos:tuple(self.generate_neighborsof(pos)) 
                               for pos in self.positions}
            return self.neighbors[position]
    # ---
    
    def generate_neighborsof(self, position, log=True):
        'Generate the coordinates of the neighboring cells.'
        # Calculate the neighbors
        for neighbor_pos in self.neighborhood:
            yield tuple((position[i] + neighbor_coord)
                            for i,neighbor_coord in enumerate(neighbor_pos))
    # ---
            
    @property
    def totalcells(self):
        'From the dimensions, calculate the total number of cells.'
        return reduce((lambda x,y: x*y), self.dimensions)
    # ---
    
    def next_position(self, current_position):
        'Fetch the next position from the current one.'
        # It is easier to work with the reverse
        reversed_next = [x for x in reversed(current_position)]
        # Try to increment the current position
        # Starting from the last coordinate
        for i, dim_size in enumerate(reversed(self.dimensions)):
            if reversed_next[i] >= dim_size-1:
                # Cannot increment, reset position
                reversed_next[i] = 0
            else:
                reversed_next[i] += 1
                break
        else:
            # If no position was incremented, 
            return None

        # Flip again to recover the actual next
        return tuple(reversed(reversed_next))
    # ---
    
    @property
    def positions(self):
        try:
            # check if positions have already been gnerated
            return self._positions
        except AttributeError:
            # Generate positions
            self._positions = tuple(self.enumpositions())
            return self.positions

    def enumpositions(self):
        'Enumerate all the positions of the automata.'
        # Initialize...
        current = tuple(0 for dim in self.dimensions)
        while True:
            # Check for a valid position
            if current:
                yield current
            else: 
                break  # No more items
            # Fetch next position
            current = self.next_position(current)
    # ---
    
    def linearize(self, coordinates):
        '''Linearize the given coordinates.
        
        The cells of the automata are stored in a 1D list,
        so, this method returns the linear index for the 
        multidimensional coordinate.
        
        For 2D (rows, cols), the i,j position is:
            j + i*rows.
        
        For 3D (rows, cols, depth) the i,j,k position is:
            k + j*depth + i*cols*depth.
        
        For N dimensions, (D0, D1, D2, ... D[N-1]) the (x0, x1, ... x[N-1]) position is:
            x[N-1] 
            + x[N-2] * D[N-1] 
            + x[N-3] * D[N-2]*D[N-1] 
            + ... 
            + x0 * (D1*D2...*D[N-1])
        '''
        dims = len(self.dimensions)
        # Need to get an item from a 1D array 
        # given it's coordinates as an n dimensional tuple
        multiplier = 1
        linear_position = 0
        for i, dim_size in reversed(list(enumerate(self.dimensions))):
            # Add the weighted coordinate
            linear_position += multiplier*coordinates[i]
            # Calculate the new multiplier
            multiplier *= dim_size
            
        return linear_position
        
    
    def at(self, coordinates):
        '''Get the cell at the given coordinates.
        
        For 2D (rows, cols), the i,j position is:
            j + i*rows.
        
        For 3D (rows, cols, depth) the i,j,k position is:
            k + j*depth + i*cols*depth.
        
        For N dimensions, (D0, D1, D2, ... D[N-1]) the (x0, x1, ... x[N-1]) position is:
            x[N-1] 
            + x[N-2] * D[N-1] 
            + x[N-3] * D[N-2]*D[N-1] 
            + ... 
            + x0 * (D1*D2...*D[N-1])
        '''
        # Auto wrapping of the coordinates
        coordinates = tuple((c+self.dimensions[i]) % self.dimensions[i] 
                                for i,c in enumerate(coordinates))
        
        try:
            return self.cells[self.tolinear[coordinates]]
        except AttributeError:
            # Linear coordinates cache has not been calculated
            self.tolinear = {coords:self.linearize(coords) for coords in self.positions}
            return self.at(coordinates)
        except KeyError as e:
            raise IndexError(f'Coordinates {coordinates} out of range.') from e
    # ---
    
    def step(self):
        'Move one time step forward.'
        self.cells = self.applyrule(self.cells)
    # ---
    
    def applyrule(self, grid):
        'Apply automaton rule to the given grid of cells.'
        # Create a new grid
        newgrid = [cell for cell in self.cells]
        # Apply the rule
        for i,coords in enumerate(self.positions):
            cell_state = self.at(coords)
            cell_neighbors = (self.at(n_coords) 
                                  for n_coords in self.neighborsof(coords))
            # Calculate the new state
            newgrid[i] = self.rule(coords, cell_state, cell_neighbors)
            
        # Return an immutable new grid
        return tuple(newgrid)
    # ---
# --- CellularAutomaton

In [711]:
life = CellularAutomaton(dimensions=(20,20))
print(life)
for _ in range(100):
    life.step()
print(life)

...#.#.#.#.##..#.##.
.#....##.....#..####
..###########..#..#.
.#.#.###.###.#.##...
..#.##..##.###.##.#.
##..#####.#...###.#.
###.##...#.##.###...
..###..#.#..###...#.
#..##....##.#.#.###.
#####.#.##.......#.#
..#.#####.#.#.###...
...##.##########...#
..#....#.#.##.#.#.##
#...#..#...###..#.##
#...##.#.#####.....#
.##.###...#.#...##..
###.....#..##.#..#..
#.#####.#....##..#..
.#.#####.....###.##.
...#..###..##.......

.........#.#........
..........#.........
.....###............
....#..#............
....###....##.......
.....#....###..#....
....................
....................
....................
........#..#........
.........###..##....
..........#...##....
....................
....................
....................
....................
........##..........
.......#............
....................
...........#........



In [712]:
class WolframCA(CellularAutomaton):
    'Instantiation of the CellularAutomata for a 1D automton'
    
    neighborhood = ( (-1,), (0,), (1,))
    
    def __init__(self, length=50, rule=(0, 1, 0, 1, 1, 0, 1, 0)):
        'Create the automaton with the specified dimensions.'
        # Initialize grid
        self.dimensions=(length,)
        self.cells = [0 for _ in range(length)]
        self.cells[length//2] = 1
        # Parse other arguments
        self.ruledef=rule
    # ---
    
    def process_neighborhood(self, neighbors):
        'Convert neighborhood to integer to apply rule.'
        left, current, right = neighbors
        return left + current*2 + right*4
    
    def rule(self, cell_coords, cell, neighbors):
        'Get next state according to the cell and neighbors'
        i = cell_coords[0]
        grid_dim = self.dimensions[0]
        return self.ruledef[self.process_neighborhood(neighbors)]
    # ---
    
    def __str__(self):
        return ''.join( '#' if cell else '.' for cell in self.state )
    # ---
    
    @property
    def state(self):
        cells = self.dimensions[0]
        state = [ self.at((i,)) for i in range(cells) ]
        return state
    # ---       
    

In [713]:
w = WolframCA(length=100, rule=(0, 1, 1, 0, 1, 0, 0, 0))
for _ in range(50):
    print(w)
    w.step()
    


..................................................#.................................................
.................................................###................................................
................................................#...#...............................................
...............................................###.###..............................................
..............................................#.......#.............................................
.............................................###.....###............................................
............................................#...#...#...#...........................................
...........................................###.###.###.###..........................................
..........................................#...............#.........................................
.........................................###.............###...............................

In [759]:
class DiffusionCA(CellularAutomaton):
    'Instantiation of the CellularAutomata with a diffusion rule'
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.diff_constant = kwargs.get('diff_constant', 0.0001)
    
    def rule(self, cell_coords, cell, neighbors):
        'Get next state according to the cell and neighbors'
        laplacian = sum(neighbors) - 4*cell
        return cell + self.diff_constant * laplacian
    # ---
    
    def tochar(self, value):
        values = ' .,:;x#X&@EEE '
        return values[int(value / (1/10))-1]
    
    def __str__(self):
        string = ''
        for row in self.state:
            string += ''.join( self.tochar(cell) for cell in row )
            string += '\n'
        return string
    # ---       
    
    

In [767]:
d = DiffusionCA(dimensions=(10, 10), 
                initialdensity=0.4, 
                diff_constant=0.000001)
print(d)

@  @ @@  @
@@@ @   @ 
  @@      
@@@@  @@ @
@    @ @ @
          
 @   @ @ @
 @@  @  @ 
@@     @ @
  @@ @  @ 



In [769]:
for _ in range(50000): 
    d.step()
print(d)
print(d.state)

&..@.&&  @
&@@.&   X.
.,E@     .
@@@&  &&.&
@.   X.&.&
.     . . 
 &   X X X
.@&  X. @.
&@.    &.&
..&&.&. &.

[[0.9824388807146607, 0.26515167019885755, 0.26507119232965715, 1.0229160249469489, 0.2578243985759625, 0.9700246076772839, 0.9219056704244151, 0.16126506308050384, 0.16769741928656423, 1.017640719720468], [0.9848786073964488, 1.0320548046106117, 1.0354943059210338, 0.26944011694147896, 0.976758206333615, 0.1638353314412791, 0.11674805706916307, 0.11569702048003822, 0.882898396843185, 0.21385991594088397], [0.26726571431630514, 0.35735690226591277, 1.1179041364308162, 1.0670153728070577, 0.16912461784138944, 0.11814932112959568, 0.1154526068298976, 0.1598321074413125, 0.16525201664382322, 0.2151140910289963], [1.0264290862615373, 1.030063182488716, 1.0260733991292066, 0.9739260911355978, 0.16120401443964522, 0.11654783098968677, 0.9610322709851684, 0.9254341992590666, 0.21077665033997975, 0.9780766962986458], [1.0175419259088412, 0.21346692318173924, 0.16164893368733674, 0.114

Now, we add these automata to the 

In [283]:
list( enumpositions((4,4)) )

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3)]

In [289]:
list( enumpositions((2,2,2,2)) )

[(0, 0, 0, 0),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 1, 1),
 (0, 1, 0, 0),
 (0, 1, 0, 1),
 (0, 1, 1, 0),
 (0, 1, 1, 1),
 (1, 0, 0, 0),
 (1, 0, 0, 1),
 (1, 0, 1, 0),
 (1, 0, 1, 1),
 (1, 1, 0, 0),
 (1, 1, 0, 1),
 (1, 1, 1, 0),
 (1, 1, 1, 1)]

In [248]:
g = (1 for i in range(2))

In [251]:
next(g)

StopIteration: 

In [393]:
dimensions=(5,5)
coordinates=(-1,-1)
tuple((c+dimensions[i]) % dimensions[i] 
                                for i,c in enumerate(coordinates))

(4, 4)

In [395]:
position = (0,0)
neighbor_pos = (-1, -1)
tuple((position[i] + neighbor_coord) for i,neighbor_coord in enumerate(neighbor_pos))

(-1, -1)

 ' .,:;x#X&@'